<a href="https://colab.research.google.com/github/Alpinsil/fake--news-detection/blob/master/Training_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install Sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 5.0 MB/s 


In [2]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

In [3]:
np.random.seed(500)

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
# Membuat Stemmer
factory = StemmerFactory()
stemmer = factory.create_stemmer()

# Label Encoder use to Encode target labels with value between 0 and n_classes-1
Encoder = LabelEncoder()

# TfidfVectorizer Convert a collection of raw documents to a matrix of TF-IDF features.
Tfidf_vect = TfidfVectorizer()

In [5]:
data_train = pd.read_csv('./Data_latih.csv')

In [6]:
data_train

,ID,label,tanggal,judul,narasi,nama file gambar
0,71,1,17-Aug-20,Pemakaian Masker Menyebabkan Penyakit Legionna...,A caller to a radio talk show recently shared ...,71.jpg
1,461,1,17-Jul-20,Instruksi Gubernur Jateng tentang penilangan ...,Yth.Seluruh Anggota Grup Sesuai Instruksi Gube...,461.png
2,495,1,13-Jul-20,Foto Jim Rohn: Jokowi adalah presiden terbaik ...,Jokowi adalah presiden terbaik dlm sejarah ban...,495.png
3,550,1,8-Jul-20,"ini bukan politik, tapi kenyataan Pak Jokowi b...","Maaf Mas2 dan Mbak2, ini bukan politik, tapi k...",550.png
4,681,1,24-Jun-20,Foto Kadrun kalo lihat foto ini panas dingin,Kadrun kalo lihat foto ini panas dingin . .,681.jpg
...,...,...,...,...,...,...
4226,999287,1,17-Jan-16,: KPK Dilarang Membawa Brimob Bersenjata Masuk...,KPK Dilarang Membawa Brimob Bersenjata Masuk G...,999287.jpg
4227,999472,1,20-Aug-20,Foto pejabat keuangan dibawah palu arit,Jangan mau dialihkan kepada pakaian adat… FOKU...,999472.png
4228,999501,1,17-Aug-20,Gambar Denny Siregar Musuh Warga Tasikmalaya d...,Akhir nya bisa terkenal bang denny,999501.jpg
4229,999848,1,21-Jul-20,Kaesang: Bapak Saya dengan Kesederhaan Bisa Ni...,Bapak Saya dengan Kesederhaan Bisa Nipu Rakyat...,999848.jpg


In [7]:
data_train['label'].value_counts()

1    3465
0     766
Name: label, dtype: int64

In [8]:
# Dari cell sebelumnya terlihat jelas bahwa dataset kita sangat tidak balance
# Untuk membuat datasetnya balance

# Pilih dataset dengan label 1 dan lakukan randomisasi untuk setiap baris datanya
false_news = data_train[data_train['label'] == 1].sample(frac=1)

# Concat dataset berlabel 1 yang telah dipilih dengan dataset berlabel 0
# dimana jumlah dataset berlabel 1 yang digabungkan sejumlah banyak dataset berlabel 0 + 200
true_fact = data_train[data_train['label'] == 0]
df = true_fact.append(false_news[:len(true_fact) + 200])

df

,ID,label,tanggal,judul,narasi,nama file gambar
19,6775,0,25-Oct-17,Sandi Sarankan Warga Ngadu ke Kelurahan Saja,Astaga! Sandiaga Sarankan Warga Ngadu ke Kelur...,6775.jpg
21,7038,0,13-Jul-17,"Irina, Istri Ahli IT Hermansyah disebut Mantan...","Irina, Istri Ahli IT Hermansyah disebut Mantan...",7038.jpg
22,7214,0,8-Mar-17,"Nezar Patria adalah Kader PKI, Ikut Rapat Seti...","Nezar Patria adalah Kader PKI, Ikut Rapat Seti...",7214.jpg
55,13483,0,17-Jul-18,Coach Indra Sjafri: Saya Pernah 1 Tahun Tak Di...,Kendati sukses mengantarkan Timnas Indonesia U...,13483.png
58,13907,0,28-Apr-18,Kronologis Polisi Berondong Sedan Terobos Razi...,"Innalillah… Hanya Karena Lalai Terobos Razia, ...",13907.png
...,...,...,...,...,...,...
3514,825739,1,5-Dec-17,Akun Twitter Kementerian Luar Negeri RI Unggah...,“Akun twitter Kemlu RI posting film porno.. ko...,825739.jpg
1134,264116,1,3-Feb-20,They call it disinfection?,They call it disinfection? Indonesia… no words.,264116.png
693,160657,1,31-Agu-18,Omongan Bin Masak Ngak DIperhatikan,“Omongan Bin Masak Ngak DIperhatikan.. Ngeri N...,160657.jpg
1649,376033,1,4-Feb-17,KTP Pemilih Digandakan Untuk Orang Yang Sama B...,"Ditemukan penggandaan KTP untuk pilkada, hati-...",376033.png


In [9]:
# menggunakan fitur narasi dalam melakukan prediksi terhadap label
feature = df['narasi']
label = df['label']

In [10]:
# Mengubah semua huruf pada setiap baris menjadi huruf kecil dan
# melakukan stemming pada setiap baris
lower = [stemmer.stem(row.lower()) for row in feature]

# Hasil stem dan lower
lower[:5]

['astaga sandiaga saran warga ngadu ke lurah saja tak perlu ke balai kota',
 'irina istri ahli it hermansyah sebut mantan psk',
 'nezar patria adalah kader pki ikut rapat tiap malam di istana telah jam 8 malam demikian tuduh dari alfian tandjung',
 'kendati sukses antar timnas indonesia u-19 juara piala aff u-19 2013 dan lolos ke putar final piala asia u-19 2014',
 'innalillah hanya karena lalai terobos razia mobil isi satu keluarga tembak polisi satu orang tewas']

In [11]:
# Melakukan tokenisasi untuk setiap baris dataset
tokens = [word_tokenize(element) for element in lower]

# Hasil tokenisasi setiap baris
tokens[:5]

[['astaga',
  'sandiaga',
  'saran',
  'warga',
  'ngadu',
  'ke',
  'lurah',
  'saja',
  'tak',
  'perlu',
  'ke',
  'balai',
  'kota'],
 ['irina', 'istri', 'ahli', 'it', 'hermansyah', 'sebut', 'mantan', 'psk'],
 ['nezar',
  'patria',
  'adalah',
  'kader',
  'pki',
  'ikut',
  'rapat',
  'tiap',
  'malam',
  'di',
  'istana',
  'telah',
  'jam',
  '8',
  'malam',
  'demikian',
  'tuduh',
  'dari',
  'alfian',
  'tandjung'],
 ['kendati',
  'sukses',
  'antar',
  'timnas',
  'indonesia',
  'u-19',
  'juara',
  'piala',
  'aff',
  'u-19',
  '2013',
  'dan',
  'lolos',
  'ke',
  'putar',
  'final',
  'piala',
  'asia',
  'u-19',
  '2014'],
 ['innalillah',
  'hanya',
  'karena',
  'lalai',
  'terobos',
  'razia',
  'mobil',
  'isi',
  'satu',
  'keluarga',
  'tembak',
  'polisi',
  'satu',
  'orang',
  'tewas']]

In [12]:
# train_test_split digunakan untuk memecah dataset menjadi 2 bagian
# X_train dan y_train mewakili data yang akan dilakukan pada fitting model(Training model)
# X_test dan y_test  mewakili data yang akan dilakukan pada evaluasi model
X_train, X_test, y_train, y_test = train_test_split(tokens, label, test_size=0.2, stratify=label)

In [13]:
# Melihat ukuran data latih dan data uji
print('X_train : ', len(X_train))
print('X_test : ', len(X_test))

X_train :  1385
X_test :  347


In [14]:
y_train = Encoder.fit_transform(y_train)
y_test = Encoder.fit_transform(y_test)

y_train

array([1, 0, 1, ..., 0, 1, 0])

In [15]:
Tfidf_vect.fit(["".join(row) for row in X_train])

TfidfVectorizer()

In [16]:
X_train_Tfidf = Tfidf_vect.transform([" ".join(row) for row in X_train])
X_test_Tfidf = Tfidf_vect.transform([" ".join(row) for row in X_test])

In [17]:
# Classifier - Algorithm - SVM
# fitting/training datasets pada algoritma SVM(Support Vector Machine)
SVM = svm.SVC(C=1.0, kernel='linear', degree=1, gamma="auto", verbose=True)
SVM.fit(X_train_Tfidf, y_train)  # predict the labels on validation dataset

# Menggunakan metrics accuracy untuk melihat performa model
predictions_SVM = SVM.predict(X_test_Tfidf)
print("SVM Accuracy Score -> ", accuracy_score(predictions_SVM, y_test)*100)

[LibSVM]SVM Accuracy Score ->  59.94236311239193


In [18]:
rf = RandomForestClassifier()
rf.fit(X_train_Tfidf, y_train)

prediction_rf = rf.predict(X_test_Tfidf)
print("RandomForest Accuracy Score -> ", accuracy_score(prediction_rf, y_test)*100)

RandomForest Accuracy Score ->  56.77233429394812


In [23]:
import pickle
#save model
filename = 'finalized_model.pkl'
pickle.dump(prediction_rf, open(filename, 'wb'))